In [2]:
#with this code, the goal is to predict the distribution of components within a mixture that is in liquid-liquid equilibrium
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import least_squares
from thermo.unifac import UNIFAC, UFSG, UFIP
from thermo.chemical import Chemical
from ugropy import Groups
import phasepy
from phasepy import mixture, component, virialgamma, preos

In [ ]:
#first we define the system conditions in which we are operating
T=25+273.15   #temperature in Kelvin
P=1.01325     #pressure in bar
#next, we need to define the components in our experiments
components=['acetic acid','hydrogen peroxide','water','diisopropyl ether']
components_initial_values=[[2.6123,'mass'],[4.125,'mass'],[12.375,'mass'],[17.8883,'mass']]  #list of values of each component, which can be given in 'mass' or 'moles'


In [ ]:
def get_molar_fractions_zi(components,components_initial_values,T,P):
    components_moles=[]
    for i in range(len(components_initial_values)):
        if components_initial_values[i][1]=='moles':
            components_moles.append(components_initial_values[i][0])
        elif components_initial_values[i][1]=='mass':
            component_molar_mass=components[i].MW
            components_moles.append(components_initial_values[i][0]/component_molar_mass)
        else:
            raise ValueError("invalid unit for component initial values. Use 'mass' or 'moles'")
    total_moles=sum(components_moles)
    molar_fraction_zi=[moles/total_moles for moles in components_moles]
    return molar_fraction_zi

In [ ]:
def get_components_properties(components,T,P):
    #this function will return a list containing the properties of each component that are needed to define a component using phasepy
    components_properties=[]
    for i in range(len(components)):
        component=Chemical(components[i],T=T,P=P)
        properties=[component.Tc,component.Pc/100000,component.Zc,component.Vc*1000000,component.omega,component.Psat.Antoine_AB_low]
        component_name=components[i]
        component_subgroup_names=Groups(component_name)
        sub = component_subgroup_names.unifac.subgroups
        properties.append(sub)
        components_properties.append(properties)
    return components_properties



In [ ]:
#Now that we have the functions to facilitate obtaining the properties neede to define the liquid-liquid equilibrium, we can start defining our system
